In [1]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

Using TensorFlow backend.


In [14]:
#https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/

In [2]:
# fix random seed for reproducibility
numpy.random.seed(7)

In [5]:
x_train = pandas.read_csv('x_train.csv')
y_train = pandas.read_csv('y_train.csv')

In [10]:
x_train.shape , y_train.shape

((164, 15), (164, 12))

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15
0,0.160005,0.056003,2.339871e-01,-0.264448,-0.320224,-0.286377,-0.295267,-0.755286,0.870381,0.929089,0.437852,0.556852,0.057507,-0.423055,15.602151
1,0.205339,0.383323,-1.151125e-01,-0.170888,-0.137041,-0.145931,-0.605950,1.019717,1.078425,0.587188,0.706188,0.206843,-0.273719,0.149336,15.452815
2,0.814702,0.316267,2.604908e-01,0.294338,0.285448,-0.174571,1.451096,1.509804,1.018567,1.137567,0.638222,0.157660,0.580715,0.431379,15.021436
3,0.586815,0.531039,5.648865e-01,0.555997,0.095977,1.721645,1.780353,1.289116,1.408116,0.908770,0.428208,0.851264,0.701928,0.270549,14.750887
4,0.654164,0.688011,6.791216e-01,0.219102,1.844770,1.903477,1.412240,1.531241,1.031895,0.551333,0.974388,0.825052,0.393673,0.123125,14.627763
5,0.575281,0.566391,1.063715e-01,1.732039,1.790747,1.299510,1.418510,0.919164,0.438603,0.861658,0.712322,0.280943,0.010394,-0.112730,14.740493
6,-0.665711,-1.125731,4.999368e-01,0.558644,0.067408,0.186408,-0.312938,-0.793500,-0.370445,-0.519781,-0.951160,-1.221708,-1.344833,-1.232102,15.972596
7,-0.800564,0.825104,8.838115e-01,0.392575,0.511575,0.012229,-0.468333,-0.045278,-0.194613,-0.625992,-0.896541,-1.019666,-0.906935,0.325167,15.647429
8,0.728514,0.787222,2.959849e-01,0.414985,-0.084361,-0.564922,-0.141867,-0.291203,-0.722582,-0.993131,-1.116255,-1.003525,0.228577,-0.096590,15.744018
9,0.794958,0.303722,4.227218e-01,-0.076624,-0.557186,-0.134131,-0.283467,-0.714845,-0.985394,-1.108519,-0.995788,0.236314,-0.088853,0.007737,15.736282


In [11]:
trainX = numpy.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
trainX.shape()

ValueError: Must pass 2-d input